In [1]:
import os
import pickle
from collections import Counter

# Загрузка данных

In [2]:
os.chdir('..')

In [3]:
names = ['arab', 'juncea', 'nigra', 'rapa']

In [4]:
Xs = {}
ys = {}

In [5]:
for name in names[1:]:
    pickle_filename = os.path.join('data', f'X_y_{name}.pkl')
    
    if os.path.exists(pickle_filename):
        try:
            with open(pickle_filename, 'rb') as f:
                data = pickle.load(f)
            
            X = data['X']
            y = data['y']
            
            Xs[name] = X
            ys[name] = y
                    
        except Exception as e:
            print(f"Error loading {name}: {e}")
    else:
        print(f"File for {name} not found.")

# Загрузка модели

In [6]:
file_path = 'models_files/random_forest_balanced_01.pkl'
clf = None
if os.path.exists(file_path):
    with open(file_path, 'rb') as f:
        clf = pickle.load(f)
else:
    print("No file")

# Предсказание

In [ ]:
for name in names[1:]:
    X = Xs[name]
    y = ys[name]
    y_pred = clf.predict(X)
        
    for i in range(len(X)):
        if y_pred[i] == 'LTR':
            print(i)
    counter = Counter(y_pred)
    print(counter)
    

Для проверки результатов руками

In [ ]:
X = Xs['juncea']
y = ys['juncea']
y_pred = clf.predict(X)
    
for i in range(len(X)):
    if y_pred[i] == 'LTR':
        print(i)
counter = Counter(y_pred)
print(counter)

In [ ]:
small_components_all = {}
small_components_families_all = {}
largest_component_all = {}
largest_component_families_all = {}

# Функция для загрузки данных из файла
def load_data_from_file(filename):
    try:
        with open(filename, 'rb') as f:
            return pickle.load(f)
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        return None

# Процесс получения меток для маленьких компонент
def process_small_components(name):
    small_components_families_filename = os.path.join('data', f'small_components_families_{name}.pkl')
    if os.path.exists(small_components_families_filename):
        data = load_data_from_file(small_components_families_filename)
        if data:
            small_components_families_all[name] = data['families']
            small_components_all[name] = data['components']
        return True
    return False

# Процесс получения кластеров и меток для большой компоненты
def process_largest_component(name):
    largest_component_filename = os.path.join('data', f'largest_component_clusters_{name}.pkl')
    if os.path.exists(largest_component_filename):
        data = load_data_from_file(largest_component_filename)
        if data:
            largest_component = data['clusters']
            largest_component_families = data['families']
            
            largest_component_all[name] = largest_component
            largest_component_families_all[name] = largest_component_families
        return True
    return False
    
    

name = 'juncea'
if not process_small_components(name):
    print(f"File for {name} (small components families) not found.")

if not process_largest_component(name):
    print(f"File for {name} (largest component) not found.")
    

small_components = small_components_all['juncea']
largest_component = largest_component_all['juncea']
print(len(small_components))
print(len(X))
print(list(largest_component[6810 - len(small_components)]))

In [33]:
import pandas as pd
from funcs.parsing import parse_fasta

filename_name = 'data_juncea/graph_collapse_nodes_b_juncea_full.txt'
filename_fasta = 'data_juncea/seq_sv_big.fasta'
output_dir = 'data'
def load_info(filename_name, filename_fasta, name):
    node2name = pd.read_csv(filename_name, sep='\t')
    node2name = node2name.drop_duplicates(subset=['node'], keep='first')
    node2name_dict = node2name.set_index('node')['name'].to_dict()
    
    fasta_file_path = filename_fasta
    name2seq = parse_fasta(fasta_file_path)
    
    # Сохраняем данные в pickle файл
    data_filename = os.path.join(output_dir, f'dicts_{name}.pkl')
    if not os.path.exists(data_filename):
        with open(data_filename, 'wb') as f:
            pickle.dump({
                'node2name_dict': node2name_dict,
                'name2seq': name2seq
            }, f)
    
    return node2name_dict, name2seq

node2name_dict, name2seq = load_info(filename_name, filename_fasta, name)


In [ ]:
for node in list(largest_component[6810 - len(small_components)]):
    print(node2name_dict[node])

In [ ]:
name2seq['SVgr_10_id_58266|5485']